# Preprocessing
This file handles all preprocessing of the original file 'safe_links_all.gz'

1. drop all non-image files
2. retrieve comments for posts (currently disabled, due to memory leak) 
3. downloading corresponding image, associating with hash of url
4. converting to fasttext format

In [8]:
import sys
import gzip
from collections import namedtuple
import requests
import traceback
import hashlib
import praw
import wget
from praw.models import MoreComments
import logging
import datetime
logging.basicConfig(format='%(asctime)s %(message)s', datefmt="%Y-%m-%d %H:%M:%S", level=logging.INFO, stream=sys.stdout)
import os
import os.path
from multiprocessing import Pool
import gensim

2019-01-10 13:08:57 'pattern' package not found; tag filters are not available for English


In [ ]:
row = namedtuple('row_raw', ['subreddit', 'submission_title', 'submitted_link', 'comments_link', 'short_name'])
base_path = "/mnt/marcel/"

reddit = praw.Reddit(user_agent='Comment Extraction',
         client_id='BQD4SITRawON4Q', client_secret="noSAX9aSU3VRPowZnhTytXm8sOk",
         )

def retrieve_file(url, filename):
    if not os.path.isfile(filename):
        try:
            wget.download(url, filename, None)
        except:
            pass 

def retrieve_comments(link):
    try:
        comments = reddit.submission(url='https://www.reddit.com'+link).comments.list()
        comments_string = ""
        for comment in comments:
            if not isinstance(comment, MoreComments):
                comments_string += comment.body + " "
        with gzip.open(base_path+'comments/%s.gz' % os.getpid(),'at') as f:
            f.write(str([link, comments_string])+"\n")
    except:
        pass
    
with gzip.open(base_path+'safe_links_all.gz','rt') as f, gzip.open(base_path+'safe_links_imgposts.gz','wt') as fout, Pool(processes=32) as pool:
    for i, line in enumerate(f):
        line = line.replace(',null]', ',""]').replace(',null,', ',"",').replace('[null,', '["",')
        if i%1000000==0:  # total number of lines: 206669601
            logging.info(i//1000000)
        line_list = eval(line)  # convert the str(list) to a list, don't do this in production, as this has the danger of code injection
        r = row._make(line_list)
        submitted_link = r.submitted_link
        if '?' in submitted_link:  # crude way to strip url params
            submitted_link = submitted_link[:r.submitted_link.find('?', 0)]                

        if submitted_link.endswith(".jpg") or submitted_link.endswith(".png") or submitted_link.endswith(".jpeg"):
            imgurlhash = hashlib.sha256(r.submitted_link.encode('utf-8')).hexdigest()
            suffix = ".jpg"
            if submitted_link.endswith(".png"):
                suffix = ".png"
            imgurlhash += suffix

            filename = base_path+"img/"+imgurlhash
            pool.apply_async(retrieve_file, (r.submitted_link, filename))
            #pool.apply_async(retrieve_comments, (r.comments_link, ))

            outlist = list(r)
            outlist.append(imgurlhash)                
            fout.write(str(outlist)+"\n")
            
            #with gzip.open(base_path+'subreddits/%s.gz' % r.subreddit,'at') as f:
            #    f.write(r.submission_title+"\n")

            #outlist.append(comments_string)
            #foutc.write(str(outlist)+"\n")

In [9]:
row = namedtuple('row_raw', ['subreddit', 'submission_title', 'submitted_link', 'comments_link', 'short_name', 'imgurlhash'])

with open(base_path+'imgposts_fasttext.train', "wt") as ft_train, open(base_path+'imgposts_fasttext.valid', "wt") as ft_valid, gzip.open(base_path+'safe_links_imgposts.gz', "rt") as f: 
    for i, line in enumerate(f):
        line_list = eval(line)  # convert the str(list) to a list, don't do this in production, as this has the danger of code injection
        r = row._make(line_list)
        output = "__label__" + r.subreddit + " " + " ".join(gensim.utils.simple_preprocess(r.submission_title)) + "\n"
        if i%10==0:
            ft_valid.write(output)
        else:
            ft_train.write(output)